In [1]:
import graphlab

In [2]:
song_data = graphlab.SFrame('song_data.gl')

[INFO] 1450744750 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to C:\Users\Kai\AppData\Local\Dato\Dato Launcher\lib\site-packages\certifi\cacert.pem
1450744750 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to gordon_chan@yahoo.com and will expire on December 05, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-5632 - Server binary: C:\Users\Kai\AppData\Local\Dato\Dato Launcher\lib\site-packages\graphlab\unity_server.exe - Server log: C:\Users\Kai\AppData\Local\Temp\graphlab_server_1450744750.log.0
[INFO] GraphLab Server Version: 1.7.1


In [3]:
graphlab.canvas.set_target('ipynb')

In [10]:
#Computer unique user for each artist
kwest = song_data[song_data['artist']=='Kanye West']
len(kwest['user_id'].unique())

2522

In [11]:
ffighters = song_data[song_data['artist']=='Foo Fighters']
len(ffighters['user_id'].unique())

2055

In [12]:
tswift = song_data[song_data['artist']=='Taylor Swift']
len(tswift['user_id'].unique())

3246

In [13]:
lgaga = song_data[song_data['artist']=='Lady GaGa']
len(lgaga['user_id'].unique())

2928

In [14]:
#groupby-aggregate to find moust and least popular
popular = song_data.groupby(key_columns='artist', 
                            operations={'total_count': graphlab.aggregate.SUM('listen_count')},)


In [17]:
popular.sort('total_count', ascending=False)

artist,total_count
Kings Of Leon,43218
Dwight Yoakam,40619
Björk,38889
Coldplay,35362
Florence + The Machine,33387
Justin Bieber,29715
Alliance Ethnik,26689
OneRepublic,25754
Train,25402
The Black Keys,22184


In [18]:
popular.sort('total_count', ascending=True)

artist,total_count
William Tabbert,14
Reel Feelings,24
Beyoncé feat. Bun B andSlim Thug ...,26
Diplo,30
Boggle Karaoke,30
harvey summers,31
Nâdiya,36
Kanye West / Talib Kweli/ Q-Tip / Common / ...,38
Aneta Langerova,38
Jody Bernal,38


In [19]:
#train data
train_data, test_data = song_data.random_split(.8, seed=0)

In [20]:
personalize_model = graphlab.item_similarity_recommender.create(train_data,
                                                               user_id='user_id',
                                                               item_id='song')

PROGRESS: Recsys training: model = item_similarity
PROGRESS: Warning: Ignoring columns song_id, listen_count, title, artist;
PROGRESS:     To use one of these as a target column, set target = <column_name>
PROGRESS:     and use a method that allows the use of a target.
PROGRESS: Preparing data set.
PROGRESS:     Data has 893580 observations with 66085 users and 9952 items.
PROGRESS:     Data prepared in: 1.19233s
PROGRESS: Computing item similarity statistics:
PROGRESS: Computing most similar items for 9952 items:
PROGRESS: +-----------------+-----------------+
PROGRESS: | Number of items | Elapsed Time    |
PROGRESS: +-----------------+-----------------+
PROGRESS: | 1000            | 1.9225          |
PROGRESS: | 2000            | 2.01464         |
PROGRESS: | 3000            | 2.0977          |
PROGRESS: | 4000            | 2.17826         |
PROGRESS: | 5000            | 2.25631         |
PROGRESS: | 6000            | 2.33237         |
PROGRESS: | 7000            | 2.40642         |


In [21]:
#list of test users
subset_test_users = test_data['user_id'].unique()[0:10000]

In [23]:
#recommend 1 song for each test user
recommend = personalize_model.recommend(subset_test_users, k=1)

PROGRESS: recommendations finished on 1000/10000 queries. users per second: 1627.97
PROGRESS: recommendations finished on 2000/10000 queries. users per second: 1743.47
PROGRESS: recommendations finished on 3000/10000 queries. users per second: 1776.18
PROGRESS: recommendations finished on 4000/10000 queries. users per second: 1786.98
PROGRESS: recommendations finished on 5000/10000 queries. users per second: 1801.61
PROGRESS: recommendations finished on 6000/10000 queries. users per second: 1813.14
PROGRESS: recommendations finished on 7000/10000 queries. users per second: 1820.52
PROGRESS: recommendations finished on 8000/10000 queries. users per second: 1821.73
PROGRESS: recommendations finished on 9000/10000 queries. users per second: 1829.9
PROGRESS: recommendations finished on 10000/10000 queries. users per second: 1829.26


In [24]:
recommend.head()

user_id,song,score,rank
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Cuando Pase El Temblor -Soda Stereo ...,0.0194504525792,1
c067c22072a17d33310d7223d7b79f819e48cf42 ...,Grind With Me (ExplicitVersion) - Pretty Ricky ...,0.0459424433009,1
f6c596a519698c97f1591ad89f540d76f6a04f1a ...,Hey_ Soul Sister - Train,0.0249187712484,1
696787172dd3f5169dc94deef97e427cee86147d ...,Senza Una Donna (WithoutA Woman) - Zucchero / ...,0.0170265780731,1
3a7111f4cdf3c5a85fd4053e3cc2333562e1e0cb ...,Heartbreak Warfare - JohnMayer ...,0.032073765364,1
532e98155cbfd1e1a474a28ed96e59e50f7c5baf ...,Jive Talkin' (AlbumVersion) - Bee Gees ...,0.0118288659232,1
ee43b175ed753b2e2bce806c903d4661ad351a91 ...,Ricordati Di Noi -Valerio Scanu ...,0.0305171277997,1
e372c27f6cb071518ae500589ae02c126954c148 ...,Fall Out - The Police,0.0819672131148,1
83b1428917b47a6b130ed471b09033820be78a8c ...,Clocks - Coldplay,0.0440762625838,1
39487deef9345b1e22881245cabf4e7c53b6cf6e ...,Black Mirror - ArcadeFire ...,0.0417737699321,1


In [25]:
popular2 = recommend.groupby(key_columns='song', 
                            operations={'total_count': graphlab.aggregate.COUNT()})

In [26]:
popular2.sort('total_count', ascending=False)

song,total_count
Undo - Björk,434
Secrets - OneRepublic,386
Revelry - Kings Of Leon,232
You're The One - DwightYoakam ...,170
Fireflies - CharttraxxKaraoke ...,122
Hey_ Soul Sister - Train,105
Horn Concerto No. 4 in Eflat K495: II. Romance ...,95
Sehr kosmisch - Harmonia,75
OMG - Usher featuringwill.i.am ...,57
Dog Days Are Over (RadioEdit) - Florence + The ...,52
